In [ ]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

def process(edl_constructor, dataframe: pd.DataFrame, pzc_she: float):
    """
    Process Monteiro data and calculate phi0-phi'
    """
    dataframe["E_SHE"] = dataframe["E vs. RHE"] - dataframe["pH"] * 59e-3
    dataframe["phi0"] = dataframe["E_SHE"] - pzc_she

    # Find unique log concentration values
    unique_logc_values = dataframe.drop_duplicates(subset="log c")["log c"].values

    # One computation for each concentration value
    processed_dfs = []

    for logc in unique_logc_values:
        df_select_logc = dataframe[dataframe["log c"] == logc].copy()
        model = edl_constructor(10**logc)

        potentials_vs_pzc = np.linspace(np.min(df_select_logc["phi0"]), 0, 50)
        sweep = model.potential_sweep(potentials_vs_pzc, tol=1e-4)

        # Interpolate the result for the phi0 values already in the dataframe
        interpolator = interp1d(potentials_vs_pzc, sweep["phi_rp"])
        df_select_logc["phi_rp"] = interpolator(df_select_logc["phi0"])
        processed_dfs.append(df_select_logc)

    return_df = pd.concat(processed_dfs)
    return_df['c'] = 10 ** return_df['log c'].values
    return return_df

In [ ]:
# for species in species_dict.keys():
#     df_list = []
#     for ph in [11, 13]:
#         df = pd.read_csv(f"data/{species}_ph_{ph}_full.csv", names=['log c', 'log j', 'E vs. RHE'], header=1)
#         df['pH'] = ph
#         df_list.append(df)
#     new_df = pd.concat(df_list, ignore_index=True)
#     new_df.to_csv(f'data/{species}.csv', index=False)

In [ ]:
from edl.models import ExplicitStern as model1
from edl.models import DoubleLayerModel as model2
from edl.constants import AU_PZC_SHE_V as PZC
species_dict = {'Li': 7, 'Na': 6, 'K': 5}
x2_dict = {'Cs': 3.5e-10, 'K': 4.1e-10, 'Na': 5.2e-10, 'Li': 5.8e-10}

In [ ]:
df_dict = {}
for species, gamma in species_dict.items():
    df = pd.read_csv(f"data/{species}.csv")
    # df = process(lambda c: model1(c, properties[0], 2, x2_dict[species]), df, pzc_she=PZC)
    df = process(lambda c: model2(c, gamma, 2), df, pzc_she=PZC)
    df_dict[species] = df

df_dict_ringe = {}
for species, gamma in species_dict.items():
    df = pd.read_csv(f"data/{species}.csv")
    df = process(lambda c: model1(c, gamma, 2, x2_dict[species]), df, pzc_she=PZC)
    # df = process(lambda c: model2(c, properties[0], 2), df, pzc_she=PZC)
    df_dict_ringe[species] = df

In [ ]:
for species, df in df_dict.items():
    df['species'] = species
all_df = pd.concat(list(df_dict.values()))

for species, df in df_dict_ringe.items():
    df['species'] = species
all_df_ringe = pd.concat(list(df_dict_ringe.values()))

# import pandas as pd
all_df.to_csv('data/au_df.csv', index=False, float_format='%10.3f')
all_df_ringe.to_csv('data/au_df_ringe.csv', index=False, float_format='%10.3f')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget

from matplotlib import rcParams
import plotting as P

rcParams["lines.linewidth"] = 0.75
rcParams["font.size"] = 8
rcParams["axes.linewidth"] = 0.5
rcParams["xtick.major.width"] = 0.5
rcParams["ytick.major.width"] = 0.5

fig = plt.figure(figsize=(6.69423, 5))
ax_li = fig.add_subplot(221)
ax_na = fig.add_subplot(222)
ax_k = fig.add_subplot(223)
ax_k_ringe = fig.add_subplot(224)

ph_fillstyle = {11: 'none', 13: 'full'}

def plot_single_species(ax, species, df, color, ph_fillstyle, legend=True):
    df = df[df['species'] == species]
    ph_11 = df.groupby('pH').get_group(11)
    ph_13 = df.groupby('pH').get_group(13)

    for ph, df in zip([11, 13], [ph_11, ph_13]):
        groupby_dict = df.groupby('c').indices
        colors = {
            list(groupby_dict.keys())[i]: color for i, color in enumerate(P.get_color_gradient(len(groupby_dict), color=color))
        }
        for c, indices in groupby_dict.items():
            select_df = df.iloc[indices]
            ax.plot(
                select_df['phi0'] - select_df['phi_rp'],
                select_df['log j'],
                marker='o',
                fillstyle=ph_fillstyle[ph],
                markeredgewidth=0.5,
                markersize=3,
                linestyle="None",
                color=colors[c],
                label=f"{c*1e3:.0f}",
            )
    if legend:
        ax.legend(frameon=False, title=r'$c_+^*$ / mM')

li_select = all_df[all_df['species'] == 'Li']
from edl import constants as C
# normfac = (2 * C.E_0 * 5e18) / (C.BETA * C.PLANCK)
p = np.polyfit(C.BETA * C.E_0 * (li_select['phi0'] - li_select['phi_rp']), li_select['log j'], deg=1)
print(p)
phi_axis = np.linspace(-2, 0, 100)
logj_axis = np.polyval(p, C.BETA * C.E_0 * phi_axis)

for ax in fig.axes:
    ax.plot(phi_axis, logj_axis, 'k-')
    ax.set_xlabel(r"$\phi_0 - \phi'$ / V")
    ax.set_ylabel(r"$\log j$ / A cm$^{-2}$")

plot_single_species(ax_li, 'Li', all_df, 'blue', ph_fillstyle=ph_fillstyle)
plot_single_species(ax_na, 'Na', all_df, 'green', ph_fillstyle=ph_fillstyle)
plot_single_species(ax_k, 'K', all_df, 'red', ph_fillstyle=ph_fillstyle)
plot_single_species(ax_k_ringe, 'Na', all_df_ringe, 'green', ph_fillstyle=ph_fillstyle, legend=False)
plot_single_species(ax_k_ringe, 'Li', all_df_ringe, 'blue', ph_fillstyle=ph_fillstyle, legend=False)
plot_single_species(ax_k_ringe, 'K', all_df_ringe, 'red', ph_fillstyle=ph_fillstyle, legend=False)

ax_li.set_xlim([-1.4, -0.9])
ax_na.set_xlim([-1.45, -0.95])
ax_k.set_xlim([-1.5, -1.0])
ax_li.set_ylim([-4.7, -2.5])
ax_na.set_ylim([-4.7, -2])
ax_k.set_ylim([-5, 0.5])
ax_k_ringe.set_xlim([-1.1, -0.6])
ax_k_ringe.set_ylim([-5.5, 1])

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget

from matplotlib import rcParams
import plotting as P

rcParams["lines.linewidth"] = 0.75
rcParams["font.size"] = 8
rcParams["axes.linewidth"] = 0.5
rcParams["xtick.major.width"] = 0.5
rcParams["ytick.major.width"] = 0.5

fig = plt.figure(figsize=(6.69423, 5))
ax_li = fig.add_subplot(221)
ax_na = fig.add_subplot(222)
ax_k = fig.add_subplot(223)

plot_single_species(ax_li, 'Li', all_df_ringe, 'blue', ph_fillstyle=ph_fillstyle)
plot_single_species(ax_na, 'Na', all_df_ringe, 'purple', ph_fillstyle=ph_fillstyle)
plot_single_species(ax_k, 'K', all_df_ringe, 'red', ph_fillstyle=ph_fillstyle)

li_select = all_df_ringe[all_df_ringe['species'] == 'Li']
from edl import constants as C
p_ringe = np.polyfit(C.BETA * C.E_0 * (li_select['phi0'] - li_select['phi_rp']), li_select['log j'], deg=1)
print(p_ringe)
phi_axis = np.linspace(-2, 0, 100)
logj_axis = np.polyval(p_ringe, C.BETA * C.E_0 * phi_axis)

ax_li.set_xlim([-0.9, -0.5])
ax_li.set_ylim([-4.7, -2.5])
ax_na.set_xlim([-0.9, -0.5])
ax_na.set_ylim([-4.7, -2])
ax_k.set_xlim([-1.1, -0.6])
ax_k.set_ylim([-5.5, 1])

for ax in fig.axes:
    ax.plot(phi_axis, logj_axis, 'k-', linewidth=0.5)
    ax.set_xlabel(r"$\phi_0 - \phi'$ / V")
    ax.set_ylabel(r"$\log j$ / A cm$^{-2}$")

plt.tight_layout()
plt.show()

In [ ]:
def plot_single_conc(ax, conc, ph, df, ph_fillstyle, tolerance=0.01):
    df = df[df['c'] > conc - tolerance]
    df = df[df['c'] < conc + tolerance]
    df = df.groupby('pH').get_group(ph)
    species_color = {'Li': 'blue', 'Na': 'purple', 'K': 'red'}

    groupby_dict = df.groupby('species').indices
    for species, indices in groupby_dict.items():
        select_df = df.iloc[indices]
        ax.plot(
            select_df['phi0'] - select_df['phi_rp'],
            select_df['log j'],
            marker='o',
            fillstyle=ph_fillstyle[ph],
            markeredgewidth=0.5,
            markersize=3,
            linestyle="None",
            color=species_color[species],
            label=f"{species} {select_df['c'].iloc[0]*1e3:.0f} mM",
        )
    ax.legend(frameon=False)

fig = plt.figure(figsize=(6, 5))
ax_conc_11 = fig.add_subplot(221)
ax_conc_ringe_11 = fig.add_subplot(222)
ax_conc_13 = fig.add_subplot(223)
ax_conc_ringe_13 = fig.add_subplot(224)

plot_single_conc(ax_conc_11, 0, 11, all_df, ph_fillstyle=ph_fillstyle, tolerance=0.02)
plot_single_conc(ax_conc_ringe_11, 0, 11, all_df_ringe, ph_fillstyle=ph_fillstyle, tolerance=0.02)
plot_single_conc(ax_conc_13, 0.1, 13, all_df, ph_fillstyle=ph_fillstyle, tolerance=0.01)
plot_single_conc(ax_conc_ringe_13, 0.1, 13, all_df_ringe, ph_fillstyle=ph_fillstyle, tolerance=0.01)

ax_conc_11.plot(phi_axis, np.polyval(p, C.BETA * C.E_0 * phi_axis), 'k-')
ax_conc_ringe_11.plot(phi_axis, np.polyval(p_ringe, C.BETA * C.E_0 * phi_axis), 'k-')
ax_conc_13.plot(phi_axis, np.polyval(p, C.BETA * C.E_0 * phi_axis), 'k-')
ax_conc_ringe_13.plot(phi_axis, np.polyval(p_ringe, C.BETA * C.E_0 * phi_axis), 'k-')

ax_conc_11.set_xlim([-1.5, -1.0])
ax_conc_11.set_ylim([-5.5, 0.7])
ax_conc_13.set_xlim([-1.5, -1.0])
ax_conc_13.set_ylim([-5.5, 0.7])
ax_conc_ringe_11.set_xlim([-1.1, -0.6])
ax_conc_ringe_11.set_ylim([-5.5, 0.7])
ax_conc_ringe_13.set_xlim([-1.1, -0.6])
ax_conc_ringe_13.set_ylim([-5.5, 0.7])

for ax in fig.axes:
    ax.set_xlabel(r"$\phi_0 - \phi'$ / V")
    ax.set_ylabel(r"$\log j$ / A cm$^{-2}$")

plt.tight_layout()
plt.show()